In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import maup
from zipfile import ZipFile
import numpy as np
import os
import shutil

# State House Special Election

In [3]:
pwd

'/Users/lilyfalk/sandbox/pdv-va/vest_va'

In [4]:
final = gpd.read_file('./va_2018_statehouse_special/va_2018_statehouse_special.shp')

In [5]:
final

,COUNTYFP,LOCALITY,VTDST,PRECINCT,HOD_DIST,S18HODDEM,S18HODREP,S18HODWRI,geometry
0,009,Amherst County,000102,New Glasgow,024,117,168,2,"POLYGON Z ((-79.04809 37.53463 0.00000, -79.04..."
1,009,Amherst County,000103,Coolwell,024,21,84,1,"POLYGON Z ((-79.08422 37.51334 0.00000, -79.08..."
2,009,Amherst County,000201,Court House,024,270,408,14,"POLYGON Z ((-79.15210 37.57883 0.00000, -79.15..."
3,009,Amherst County,000202,Temperance,024,61,140,3,"POLYGON Z ((-79.30301 37.74092 0.00000, -79.30..."
4,009,Amherst County,000301,Monroe,024,68,163,5,"POLYGON Z ((-79.19354 37.57677 0.00000, -79.19..."
...,...,...,...,...,...,...,...,...,...
74,161,Roanoke County,000108,WILDWOOD,008,335,664,3,"POLYGON Z ((-80.08046 37.32419 0.00000, -80.08..."
75,161,Roanoke County,000106,GREEN HILL,008,643,1271,3,"POLYGON Z ((-80.08835 37.28149 0.00000, -80.08..."
76,161,Roanoke County,000103,GLENVAR,008,374,835,2,"POLYGON Z ((-80.21886 37.26382 0.00000, -80.21..."
77,121,Montgomery County,000304,Precinct C-4,008,599,810,3,"POLYGON Z ((-80.30263 37.16025 0.00000, -80.30..."


In [36]:
county_dict = pd.Series(final['COUNTYFP'].values, index = final['LOCALITY']).to_dict()

## Election results

### District 8

In [288]:
df8 = pd.read_csv('./Virginia_Elections_Database__2018_House_of_Delegates_General_Election_District_8_including_precincts.csv')

In [289]:
df8.head()

,County/City,Ward,Pct,Joseph P. McNamara,"Edward Carter Turner, III",All Others,Total Votes Cast
0,NaN,NaN,NaN,Republican,Democratic,NaN,NaN
1,Craig County,-,101 - Court House,300,80,1.0,381
2,Craig County,-,201 - Ammendale Precinct,402,90,0.0,492
3,Craig County,-,301 - Givens & Reynolds Store,188,92,0.0,280
4,Craig County,-,303 - Marshalls Store,160,77,0.0,237


In [290]:
df8['County/City'].nunique()

5

In [291]:
df8.shape

(46, 7)

In [292]:
df8 = df8.iloc[1:]

In [293]:
df8 = df8.drop(['Ward'], axis = 1)

In [294]:
df8['HOD_DIST'] = '8'

In [295]:
#dont actually need this since function rename didnt work
column_dict = {'County/City': 'LOCALITY', 'Joseph P. McNamara': 'S18HODREP', 'Edward Carter Turner, III': 'S18HODDEM',
              'All Others': 'S18HODWRI',
              'Ronnie Ray Campbell': 'S18HODREP', 'Christian Renau Worth': 'S18HODDEM'}

In [296]:
df8['LOCALITY'] = df8['County/City'].rename(column_dict.get('County/City'))
df8['S18HODREP'] = df8['Joseph P. McNamara']
df8['S18HODDEM'] = df8['Edward Carter Turner, III']
df8['S18HODWRI'] = df8['All Others']
df8 = df8.drop(['County/City', 'Joseph P. McNamara', 'Edward Carter Turner, III', 'All Others'], axis = 1)

In [302]:
df8.head()

,Pct,Total Votes Cast,HOD_DIST,LOCALITY,S18HODREP,S18HODDEM,S18HODWRI
1,101 - Court House,381,8,Craig County,300,80,1.0
2,201 - Ammendale Precinct,492,8,Craig County,402,90,0.0
3,301 - Givens & Reynolds Store,280,8,Craig County,188,92,0.0
4,303 - Marshalls Store,237,8,Craig County,160,77,0.0
5,401 - Barbours Creek,198,8,Craig County,149,49,0.0


In [303]:
df8.LOCALITY.nunique()

5

In [273]:
'''
Below code messing up the locality, so stop useing it

def rename_columns(results, column_dict):
    for col in results.columns:
        print(col)
        if col in column_dict.keys():
            col_new_name = column_dict.get(col)
            results = results.rename(columns = {col:col_new_name})
            print(col_new_name)
    return results.head()
'''

In [304]:
county_dict = pd.Series(final['COUNTYFP'].values, index = final['LOCALITY']).to_dict()

In [305]:
county_dict

{'Amherst County': '009',
 'Augusta County': '015',
 'Bath County': '017',
 'Rockbridge County': '163',
 'Buena Vista City': '530',
 'Lexington City': '678',
 'Craig County': '045',
 'Montgomery County': '121',
 'Roanoke County': '161',
 'Salem City': '775'}

In [307]:
df8['LOCALITY'].value_counts()

Roanoke County       15
Salem City           12
Craig County          9
Montgomery County     8
TOTALS                1
Name: LOCALITY, dtype: int64

In [310]:
df8['COUNTYFP'] = df8['LOCALITY'].map(county_dict)

In [311]:
df8.COUNTYFP.nunique()

4

In [312]:
df8.LOCALITY.nunique()

5

In [313]:
column_list = ['S18HODREP', 'S18HODDEM', 'S18HODWRI']

In [314]:
df8['S18HODREP'] = df8['S18HODREP'].map(lambda x: str(x).replace(',', ''))
df8['S18HODDEM'] = df8['S18HODDEM'].map(lambda x: str(x).replace(',', '')) 
df8['S18HODWRI'] = df8['S18HODWRI'].map(lambda x: str(x).replace(',', ''))

In [315]:
df8['S18HODREP'] = df8['S18HODREP'].astype(str).astype(float).astype(int)
df8['S18HODDEM'] = df8['S18HODDEM'].astype(str).astype(float).astype(int)
df8['S18HODWRI'] = df8['S18HODWRI'].astype(str).astype(float).astype(int)

In [316]:
df8.groupby(['COUNTYFP']).sum()

,S18HODREP,S18HODDEM,S18HODWRI
COUNTYFP,,,
045,1666,519,1
121,3706,2955,16
161,10487,5912,23
775,5942,3906,10


### District 24

In [355]:
df24 = pd.read_csv('./Virginia_Elections_Database__2018_House_of_Delegates_Special_General_Election_District_24_including_precincts.csv')

In [356]:
df24.head()

,County/City,Ward,Pct,Ronnie Ray Campbell,Christian Renau Worth,All Others,Total Votes Cast
0,NaN,NaN,NaN,Republican,Democratic,NaN,NaN
1,Amherst County,-,102 - New Glasgow,164,114,2.0,280
2,Amherst County,-,103 - Coolwell,82,20,1.0,103
3,Amherst County,-,201 - Court House,397,261,13.0,671
4,Amherst County,-,202 - Temperance,137,59,3.0,199


In [357]:
df24['County/City'].nunique()

7

In [358]:
df24.shape

(55, 7)

In [359]:
df24 = df24.iloc[1:]

In [360]:
df24 = df24.drop(['Ward'], axis = 1)

In [361]:
df24['HOD_DIST'] = '24'

In [362]:
df24['County/City'].nunique()

7

In [364]:
df24['LOCALITY'] = df24['County/City']
df24['S18HODREP'] = df24['Ronnie Ray Campbell']
df24['S18HODDEM'] = df24['Christian Renau Worth']
df24['S18HODWRI'] = df24['All Others']
df24 = df24.drop(['County/City', 'Ronnie Ray Campbell', 'Christian Renau Worth', 'All Others'], axis = 1)

In [223]:
#df24 = rename_columns(df24, column_dict)

In [365]:
df24['COUNTYFP'] = df24['LOCALITY'].map(county_dict)

In [366]:
df24['LOCALITY'].nunique()

7

In [367]:
df24['COUNTYFP'].nunique()

6

In [368]:
df24['COUNTYFP'].value_counts()

163    19
009    11
017     9
015     8
530     3
678     3
Name: COUNTYFP, dtype: int64

In [369]:
df8.COUNTYFP.value_counts()

161    15
775    12
045     9
121     8
Name: COUNTYFP, dtype: int64

In [370]:
df24['S18HODREP'] = df24['S18HODREP'].map(lambda x: str(x).replace(',', ''))
df24['S18HODDEM'] = df24['S18HODDEM'].map(lambda x: str(x).replace(',', '')) 
df24['S18HODWRI'] = df24['S18HODWRI'].map(lambda x: str(x).replace(',', ''))

In [371]:
df24['S18HODREP'] = df24['S18HODREP'].astype(str).astype(float).astype(int)
df24['S18HODDEM'] = df24['S18HODDEM'].astype(str).astype(float).astype(int)
df24['S18HODWRI'] = df24['S18HODWRI'].astype(str).astype(float).astype(int)

In [372]:
df24.groupby(['COUNTYFP']).sum()

,S18HODREP,S18HODDEM,S18HODWRI
COUNTYFP,,,
009,1659,844,45
015,1369,465,1
017,360,227,0
163,2560,1861,17
530,385,236,0
678,287,853,4


In [373]:
df8.COUNTYFP.nunique() + df24.COUNTYFP.nunique()

10

In [374]:
df8.COUNTYFP.value_counts()

161    15
775    12
045     9
121     8
Name: COUNTYFP, dtype: int64

In [375]:
final.COUNTYFP.nunique()

10

In [376]:
final.HOD_DIST.value_counts()

024    42
008    37
Name: HOD_DIST, dtype: int64

In [377]:
final

,COUNTYFP,LOCALITY,VTDST,PRECINCT,HOD_DIST,S18HODDEM,S18HODREP,S18HODWRI,geometry
0,009,Amherst County,000102,New Glasgow,024,117,168,2,"POLYGON Z ((-79.04809 37.53463 0.00000, -79.04..."
1,009,Amherst County,000103,Coolwell,024,21,84,1,"POLYGON Z ((-79.08422 37.51334 0.00000, -79.08..."
2,009,Amherst County,000201,Court House,024,270,408,14,"POLYGON Z ((-79.15210 37.57883 0.00000, -79.15..."
3,009,Amherst County,000202,Temperance,024,61,140,3,"POLYGON Z ((-79.30301 37.74092 0.00000, -79.30..."
4,009,Amherst County,000301,Monroe,024,68,163,5,"POLYGON Z ((-79.19354 37.57677 0.00000, -79.19..."
...,...,...,...,...,...,...,...,...,...
74,161,Roanoke County,000108,WILDWOOD,008,335,664,3,"POLYGON Z ((-80.08046 37.32419 0.00000, -80.08..."
75,161,Roanoke County,000106,GREEN HILL,008,643,1271,3,"POLYGON Z ((-80.08835 37.28149 0.00000, -80.08..."
76,161,Roanoke County,000103,GLENVAR,008,374,835,2,"POLYGON Z ((-80.21886 37.26382 0.00000, -80.21..."
77,121,Montgomery County,000304,Precinct C-4,008,599,810,3,"POLYGON Z ((-80.30263 37.16025 0.00000, -80.30..."


### Districts 24 and 8 together

In [378]:
elections = []
elections.append(df24)
elections.append(df8)

In [379]:
df = pd.concat(elections, axis = 0)

In [380]:
df = df[df['LOCALITY'] != 'TOTALS']

In [381]:
def get_absentee_frac(prec_tot, county_tot, absentee_tot):
    '''A spec specifies in first line what function will do
    Returns fraction of county absentee votes for a precinct for a candidate
    
    Keyword arguments: (put inputs to funtions and say what they should be)
    precinct vote total for given candidate (int)
    county vote total for given candidate, absentee and provisional votes not included (int)
    absentee vote total for county for given candidate (int)
    
    "Pure function" bc only relies on inputs and outputs - only variables are ones that were passed as arguments
    '''
    if county_tot != 0:
        prec_frac = prec_tot/county_tot
        absentee_frac = prec_frac*absentee_tot
    else:
        absentee_frac = 0
    return absentee_frac

In [382]:
absentee_and_prov = df[(df['Pct'].map(lambda x: 'Absentee' in str(x))) | (df['Pct'].map(lambda x: 'Provisional' in str(x)))]
absentee_and_prov

,Pct,Total Votes Cast,HOD_DIST,LOCALITY,S18HODREP,S18HODDEM,S18HODWRI,COUNTYFP
10,Ab - Central Absentee Precinct,66,24,Amherst County,40,25,1,009
11,Provisional,2,24,Amherst County,1,1,0,009
18,Ab - Central Absentee Precinct,24,24,Augusta County,18,6,0,015
19,Provisional,1,24,Augusta County,1,0,0,015
27,Ab - Central Absentee Precinct,20,24,Bath County,7,13,0,017
28,Provisional,0,24,Bath County,0,0,0,017
31,Provisional,0,24,Buena Vista City,0,0,0,530
33,Ab - Central Absentee Precinct,91,24,Lexington City,27,62,2,678
34,Provisional,0,24,Lexington City,0,0,0,678
52,Ab - Central Absentee Precinct,151,24,Rockbridge County,78,73,0,163


In [383]:
absentee_and_prov[column_list].sum().sum()

2765

In [384]:
groupby_absentee_and_prov_tot = absentee_and_prov.groupby(['COUNTYFP']).sum()
groupby_absentee_and_prov_tot

,S18HODREP,S18HODDEM,S18HODWRI
COUNTYFP,,,
009,41,26,1
015,19,6,0
017,7,13,0
045,0,0,0
121,170,217,0
161,722,564,1
163,78,75,0
530,0,0,0
678,27,62,2


In [385]:
groupby_county_df_tot = df.groupby(['COUNTYFP']).sum()
df_no_absent_or_provisional = df[(df['Pct'].map(lambda x: 'Absentee' not in str(x))) & (df['Pct'].map(lambda x: 'Provisional' not in str(x)))
                                & (df['LOCALITY'] != 'TOTALS') & (df['COUNTYFP'] != '009')]
groupby_county_tot_no_absentee = df_no_absent_or_provisional.groupby('COUNTYFP').sum()

In [386]:
df_no_absent_or_provisional[column_list].sum().sum()

41071

In [387]:
groupby_county_tot_no_absentee

,S18HODREP,S18HODDEM,S18HODWRI
COUNTYFP,,,
015,1350,459,1
017,353,214,0
045,1666,519,1
121,3536,2738,16
161,9765,5348,22
163,2482,1786,17
530,385,236,0
678,260,791,2
775,5514,3601,9


In [354]:
df.fillna(0)

,County/City,Pct,Ronnie Ray Campbell,Christian Renau Worth,All Others,Total Votes Cast,HOD_DIST,LOCALITY,S18HODREP,S18HODDEM,S18HODWRI,COUNTYFP
1,Amherst County,102 - New Glasgow,164,114,2.0,280,24,Amherst County,164,114,2,009
2,Amherst County,103 - Coolwell,82,20,1.0,103,24,Amherst County,82,20,1,009
3,Amherst County,201 - Court House,397,261,13.0,671,24,Amherst County,397,261,13,009
4,Amherst County,202 - Temperance,137,59,3.0,199,24,Amherst County,137,59,3,009
5,Amherst County,301 - Monroe,159,66,5.0,230,24,Amherst County,159,66,5,009
...,...,...,...,...,...,...,...,...,...,...,...,...
40,0,8 - East Salem,0,0,0.0,334,8,Salem City,198,136,0,775
41,0,9 - South Side Hills,0,0,0.0,"1,247",8,Salem City,834,413,0,775
42,0,10 - Hidden Valley,0,0,0.0,"1,367",8,Salem City,830,536,1,775
43,0,Ab - Central Absentee Precinct,0,0,0.0,729,8,Salem City,424,304,1,775


In [388]:
df_with_absentee_reallocated = df_no_absent_or_provisional.copy()
for index, precinct_total_row_by_candidate in df_no_absent_or_provisional.iterrows():
    #print(precinct_total_row_by_candidate)
    county_fips = precinct_total_row_by_candidate['COUNTYFP']
    county_total_row_by_candidate_no_absentee = groupby_county_tot_no_absentee.loc[county_fips]
    county_absentee_and_provisional_by_candidate = groupby_absentee_and_prov_tot.loc[county_fips]
    #print(precinct_total_row_by_candidate, county_total_row_by_candidate_no_absentee, county_absentee_and_provisional_by_candidate)
    for candidate, votes in county_total_row_by_candidate_no_absentee.to_dict().items():
        prec_tot = precinct_total_row_by_candidate[candidate]
        #print('precinct total: ', prec_tot)
        county_tot = votes
        #print('county total: ', county_tot)
        absentee_tot = county_absentee_and_provisional_by_candidate[candidate]
        #print('absentee total: ', absentee_tot)
        absentee_frac = get_absentee_frac(prec_tot, county_tot, absentee_tot)
        #print('absentee votes to allocate to precinct: ', absentee_frac)
        prec_with_absentee_allocation = prec_tot + absentee_frac
        #print('new precinct total: ', prec_with_absentee_allocation)
        df_with_absentee_reallocated.loc[index, candidate] = round(prec_with_absentee_allocation) #index is row, candidate is column

In [391]:
df_with_absentee_reallocated.shape

(70, 8)

In [390]:
(df_with_absentee_reallocated.append(df[df.COUNTYFP == '009'])).shape

(81, 8)

In [392]:
final.shape

(79, 9)

In [396]:
groupby_county_final = final.groupby('COUNTYFP').sum()

In [397]:
groupby_df_with_absentee_reallocated = df_with_absentee_reallocated.groupby('COUNTYFP').sum()

In [398]:
df_absentee_reallocate_with_final = pd.merge(groupby_df_with_absentee_reallocated, groupby_county_final, on = 'COUNTYFP', how = 'inner', suffixes = ('_x', '_y'))

In [399]:
def sum_checker(df, column_list):
    list_max = 0
    for i in column_list:
        left_sum = df[i+'_x'].sum()
        right_sum = df[i+'_y'].sum()
        if (left_sum == right_sum):
            print('column: ', i, ' have the same sums')
        
        else:
            diff = abs(left_sum - right_sum)
            percent = abs(left_sum - right_sum)/left_sum
            print('column: ', i)
            print('RDH sum is ', str(left_sum))
            print('Partner sum is ', str(right_sum))
            print('their (absolute) difference is ', str(diff))
            print('As a percentage of DF1 this is ', str(round(percent*100,5)) + '%')
            if (percent>list_max):
                list_max = percent
            print('')
    print('The max difference (as a percent of the total of on DF column is: ', str(round(list_max,5)))

In [400]:
sum_checker(df_absentee_reallocate_with_final, column_list)

column:  S18HODREP
RDH sum is  38641.0
Partner sum is  26762
their (absolute) difference is  11879.0
As a percentage of DF1 this is  30.74196%

column:  S18HODDEM
RDH sum is  23596.0
Partner sum is  16934
their (absolute) difference is  6662.0
As a percentage of DF1 this is  28.2336%

column:  S18HODWRI
RDH sum is  92.0
Partner sum is  72
their (absolute) difference is  20.0
As a percentage of DF1 this is  21.73913%

The max difference (as a percent of the total of on DF column is:  0.30742
